In [1]:
from datetime import datetime
import requests
from io import BytesIO
from PIL import Image

import pandas as pd
import numpy as np

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl
from plotly.subplots import make_subplots

from resources.plotly_theme import nfl_template
from resources.heat_map import heat_map
from resources.get_nfl_data import get_pbp_data, get_team_info
from resources.team_stats import get_team_data


pio.templates['nfl_template'] = nfl_template

In [2]:
''' Parameters '''

WEEK = 9
AWAY_TEAM = 'IND'
HOME_TEAM = 'PIT'


# Process

In [3]:
''' Import Data '''

# Import
team_data = get_team_info()
pbp_data = get_pbp_data(years=[2025])

player_info = nfl.import_players()

game_data = pbp_data.loc[(pbp_data['week'] == WEEK) & 
                         (pbp_data['home_team'] == HOME_TEAM), :]

print(game_data.shape)

2025 done.
Downcasting floats.
(181, 378)


In [7]:
print(player_info.loc[player_info['gsis_id'] == '00-0036223',:].head().to_string())

          gsis_id     display_name common_first_name first_name last_name short_name football_name suffix     esb_id nfl_id    pfr_id pff_id otc_id  espn_id                              smart_id  birth_date position_group position ngs_position_group ngs_position  height  weight                                                                           headshot college_name  college_conference jersey_number  rookie_season  last_season latest_team status ngs_status ngs_status_short_description  years_of_experience pff_position pff_status  draft_year  draft_round  draft_pick draft_team
21069  00-0036223  Jonathan Taylor          Jonathan   Jonathan    Taylor   J.Taylor      Jonathan   None  TAY431618  52449  TaylJo02  57488   8781  4242335  32005441-5943-1618-c081-5dd6b2d0b829  1999-01-19             RB       RB                 RB           RB    70.0   226.0  https://static.www.nfl.com/image/upload/f_auto,q_auto/league/uwmnrhwtwug0wrgkomjk    Wisconsin  Big Ten Conference            28   

# Offense

In [4]:
team_offense = get_team_data(game_data, unit='offense')
team_offense = team_offense.round(2)

print(team_offense.to_string())

         Games  Plays  TDs  FirstDowns  RushAttempts  RushYards  RushTDs  Rush1Ds  DesignedRushPlays  DesignedRushAttempts  DesignedRushYards  QBScrambles  ScrambleYards  DesignedPassPlays  Dropbacks  PassCompletions  PassAttempts  PassYards  PassTDs  Pass1Ds  Sacks  SackYards  INTs  TFLs  Fumbles  Penalties  PenaltyYards  Penalty1Ds  Drives  Total Yards  Turnovers  PlaysAdv  PassPlays  RushPlays   EPA  RushEPA  PassEPA  Successes  RushSuccesses  PassSuccesses  EPA / Play  Rush EPA / Play  Pass EPA / Play  Success Rate  Rush Success Rate  Pass Success Rate
posteam                                                                                                                                                                                                                                                                                                                                                                                                                                              

## Rushing

In [5]:
''' Rushing '''

run_data = game_data.loc[game_data['rush'] == 1, :]


In [6]:
''' Team Rushing '''

team_rushing = run_data.groupby(['posteam', 'game_half']).aggregate(
    Plays=('rush', 'sum'),
    Attempts=('rush_attempt', 'sum'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum')
)
totals = team_rushing.groupby(['posteam']).sum()
totals.index = pd.MultiIndex.from_tuples([(i, 'Total') for i in totals.index])
team_rushing = pd.concat([team_rushing, totals]).sort_index()

team_rushing['Yds / Att'] = round(team_rushing['Yards'] / team_rushing['Attempts'], 2)
team_rushing['Success Rate'] = round((team_rushing['Successes'] / team_rushing['Attempts']) * 100, 2)
team_rushing['1D Rate'] = round((team_rushing['FirstDowns'] / team_rushing['Attempts']) * 100, 2)
team_rushing['TD Rate'] = round((team_rushing['TDs'] / team_rushing['Attempts']) * 100, 2)

print(team_rushing.to_string())


                   Plays  Attempts  Yards  TDs  FirstDowns  Successes  Yds / Att  Success Rate    1D Rate  TD Rate
posteam game_half                                                                                                 
IND     Half1       11.0      11.0   30.0  1.0         4.0        8.0       2.73     72.730003  36.360001     9.09
        Half2        7.0       7.0   24.0  0.0         2.0        4.0       3.43     57.139999  28.570000     0.00
        Total       18.0      18.0   54.0  1.0         6.0       12.0       3.00     66.669998  33.330002     5.56
PIT     Half1       10.0      10.0   31.0  1.0         4.0        6.0       3.10     60.000000  40.000000    10.00
        Half2       11.0      11.0    7.0  1.0         1.0        1.0       0.64      9.090000   9.090000     9.09
        Total       21.0      21.0   38.0  2.0         5.0        7.0       1.81     33.330002  23.809999     9.52


In [7]:
''' Player Rushing '''

by_rusher = run_data.groupby(['posteam', 'rusher']).aggregate(
    Plays=('rush', 'sum'),
    Attempts=('rush_attempt', 'sum'),
    Yards=('rushing_yards', 'sum'),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum'),
).sort_values(by=['posteam', 'Attempts'], ascending=False)

by_rusher['Yds / Att'] = round(by_rusher['Yards'] / by_rusher['Attempts'], 2)
by_rusher['Success Rate'] = round((by_rusher['Successes'] / by_rusher['Attempts']) * 100, 2)
by_rusher['EPA / Play'] = round((by_rusher['EPA'] / by_rusher['Plays']), 2)
by_rusher['1D Rate'] = round((by_rusher['FirstDowns'] / by_rusher['Attempts']) * 100, 2)
by_rusher['TD Rate'] = round((by_rusher['TDs'] / by_rusher['Attempts']) * 100, 2)

print(by_rusher.to_string())

                    Plays  Attempts  Yards  TDs  FirstDowns  Successes       EPA  Yds / Att  Success Rate  EPA / Play     1D Rate  TD Rate
posteam rusher                                                                                                                            
PIT     J.Warren     16.0      16.0   31.0  2.0         3.0        4.0 -4.220659       1.94     25.000000       -0.26   18.750000     12.5
        K.Gainwell    3.0       3.0   -1.0  0.0         1.0        2.0 -0.047559      -0.33     66.669998       -0.02   33.330002      0.0
        Co.Heyward    1.0       1.0    4.0  0.0         1.0        1.0  1.606266       4.00    100.000000        1.61  100.000000      0.0
        K.Johnson     1.0       1.0    4.0  0.0         0.0        0.0 -0.132659       4.00      0.000000       -0.13    0.000000      0.0
IND     J.Taylor     14.0      14.0   45.0  0.0         3.0        9.0  0.948251       3.21     64.290001        0.07   21.430000      0.0
        D.Jones       2.0  

In [7]:
# Define your colorscale (using a built-in sequential scale as an example)
colorscale_name = 'Greens'
colorscale = cl.sequential.__dict__[colorscale_name]
print(colorscale)

# Extract colors from the colorscale
rgb_colors = cl.colorscale_to_colors(['rgb(0,68,27)'])
print(rgb_colors)

# Convert RGB colors to hex format
hex_colors = cl.convert_colors_to_same_type(rgb_colors, colortype='hex')

print(f"Hex colors from '{colorscale_name}' colorscale:")
for hex_code in hex_colors:
    print(hex_code)

['rgb(247,252,245)', 'rgb(229,245,224)', 'rgb(199,233,192)', 'rgb(161,217,155)', 'rgb(116,196,118)', 'rgb(65,171,93)', 'rgb(35,139,69)', 'rgb(0,109,44)', 'rgb(0,68,27)']
['g']


PlotlyError: You must select either rgb or tuple for your colortype variable.

In [272]:
''' Rush by Down / Distance '''

MIN_ATTEMPTS = 2

## Group by down/distance
rusher_carries = run_data.groupby(['rusher', 'Down & Distance']).aggregate(
    Attempts=('rush_attempt', 'sum'),
    Successes=('success', 'sum'),
    player_id=('rusher_player_id', 'first'),
).reset_index()
rusher_carries['Success Rate'] = rusher_carries['Successes'] / rusher_carries['Attempts']

## Pivot
DOWNS = ['1st', '2nd & Short', '2nd & Medium', '2nd & Long', 
       '3rd & Short', '3rd & Medium', '3rd & Long', '4th & Short', '4th & Medium']
col_sort = pd.MultiIndex.from_product([DOWNS, ['Successes', 'Attempts', 'Success Rate']])

piv = rusher_carries.loc[(rusher_carries['Down & Distance'] != ''),:].pivot(
        index='rusher',
        columns=['Down & Distance'],
        values=['Attempts', 'Successes', 'Success Rate']
    )\
    .swaplevel(0, 1, axis=1).sort_index(axis=1).reset_index().set_index('rusher')\
    .reindex(columns=col_sort)

print(piv.to_string())

## Visualize Success Rate

# Filter to success rate
idx = pd.IndexSlice
sl = pd.DataFrame(piv.loc[:, idx[:, ('Success Rate')]])
sl.columns = sl.columns.get_level_values(0)

# Merge in total attempts
sl = sl.merge(
        by_rusher.reset_index().set_index('rusher')['Attempts'], 
        right_index=True,
        left_index=True,
        how='left'
    ).sort_values(by='Attempts', ascending=False)\
    .reset_index().set_index('rusher')
# piv = piv.set_index('rusher')
print(sl.to_string())

# Create annotations
annot_df = sl.copy() # Initialize with data, convert to string
for r in annot_df.index:
    print(r)
    for c in piv.columns.get_level_values(0).unique():
        attempts = piv.loc[r, (c, 'Attempts')]
        if pd.isna(attempts):
            annot_df.loc[r,c] = ''
        else:
            annot_df.loc[r, c] = f"{piv.loc[r, (c, 'Success Rate')]*100:,.1f}%<br>({piv.loc[r,(c, 'Successes')]:,.0f} / {attempts:,.0f})"

# print(annot_df.head().to_string())
# print(sl.head().to_string())
# print(sl.index)

# Heatmap

# fig = go.Figure()

cols = [col.replace(' ', '<br>') for col in sl.columns.get_level_values(0).drop('Attempts')]

fig = heat_map(
    x=cols,
    y=sl.index,
    z=sl.values.tolist(),
    text=annot_df.values.tolist(),
    title=f'Rushing <span style="color: #006D2C">Success Rate</span> by Down & Distance<br><sup>NFL Week {WEEK}: {AWAY_TEAM} @ {HOME_TEAM}</sup>',
    color_name='Success Rate'
)
fig.update_layout(
    margin=dict(t=100, l=100, r=100, b=50),
    width=900,
    height=500
)
fig.update_annotations(y=-.1)

# headshots = rusher_carries['Headshot'].tolist()
# n_players = len(headshots)
# for i in range(n_players):
#     response = requests.get(headshots[i])
#     headshot = Image.open(BytesIO(response.content))
#     fig.add_layout_image(
#         x=-.135,
#         # y=(1-(1/n_players))-((1/n_players)*i),
#         y=1.01-((1/n_players)*i),
#         sizex=1/n_players,
#         sizey=1/n_players,
#         xanchor='center',
#         yanchor='top',
#         xref='paper', 
#         yref='paper',
#         source=headshot,
#     )

fig.show()

# pio.write_image(fig, "Success Rate by Down & Distance - Colts vs. Rams.png", height=500, width=900, scale=6)

                  1st                       2nd & Short                       2nd & Medium                       2nd & Long                       3rd & Short                       3rd & Medium                       3rd & Long                       4th & Short                       4th & Medium                      
            Successes Attempts Success Rate   Successes Attempts Success Rate    Successes Attempts Success Rate  Successes Attempts Success Rate   Successes Attempts Success Rate    Successes Attempts Success Rate  Successes Attempts Success Rate   Successes Attempts Success Rate    Successes Attempts Success Rate
rusher                                                                                                                                                                                                                                                                                                                      
A.Abdullah        0.0      1.0     0.000000      

## Passing

In [8]:
''' Passing '''

pass_data = game_data.loc[game_data['pass'] == 1, :]

In [9]:
''' Team Passing '''
# NOTE - Team passing includes sacks, both attempts and yards

team_passing = pass_data.groupby(['posteam', 'game_half']).aggregate(
    Plays=('pass', 'sum'),
    Attempts=('pass_attempt', 'sum'),
    Completions=('complete_pass', 'sum'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    INTs=('interception', 'sum'),
    Sacks=('sack', 'sum'),
    SackYards=('yards_gained', lambda x: x[pass_data['sack'] == 1].sum()),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum'),
)
totals = team_passing.groupby(['posteam']).sum()
totals.index = pd.MultiIndex.from_tuples([(i, 'Total') for i in totals.index])
team_passing = pd.concat([team_passing, totals]).sort_index()
team_passing['Attempts'] = team_passing['Attempts'] - team_passing['Sacks']

team_passing['Yds / Att'] = round(team_passing['Yards'] / team_passing['Attempts'], 2)
team_passing['Success Rate'] = round((team_passing['Successes'] / team_passing['Plays']) * 100, 2)
team_passing['EPA / Play'] = round((team_passing['EPA'] / team_passing['Plays']), 2)
team_passing['1D Rate'] = round((team_passing['FirstDowns'] / team_passing['Attempts']) * 100, 2)
team_passing['TD Rate'] = round((team_passing['TDs'] / team_passing['Attempts']) * 100, 2)

print(team_passing.to_string())

                   Plays  Attempts  Completions  Yards  TDs  INTs  Sacks  SackYards  FirstDowns  Successes        EPA  Yds / Att  Success Rate  EPA / Play    1D Rate  TD Rate
posteam game_half                                                                                                                                                             
IND     Half1       24.0      20.0         11.0  108.0  0.0   1.0    1.0       -6.0         6.0        8.0 -10.139390       5.40     33.330002       -0.42  30.000000     0.00
        Half2       35.0      29.0         20.0  206.0  1.0   2.0    4.0      -23.0        13.0       19.0  -5.700649       7.10     54.290001       -0.16  44.830002     3.45
        Total       59.0      49.0         31.0  314.0  1.0   3.0    5.0      -29.0        19.0       27.0 -15.840039       6.41     45.759998       -0.27  38.779999     2.04
PIT     Half1       26.0      22.0         14.0   97.0  1.0   0.0    2.0      -12.0         6.0       11.0  -4.279295       4

In [10]:
''' Passing - QBs '''


by_passer = pass_data.groupby(['posteam', 'passer']).aggregate(
    Plays=('pass', 'sum'),
    Attempts=('pass_attempt', 'sum'),
    Completions=('complete_pass', 'sum'),
    Yards=('passing_yards', 'sum'),
    TDs=('touchdown', 'sum'),
    INTs=('interception', 'sum'),
    Sacks=('sack', 'sum'),
    SackYards=('yards_gained', lambda x: x[pass_data['sack'] == 1].sum()),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum'),
)
# totals = by_passer.groupby(['posteam']).sum()
# totals.index = pd.MultiIndex.from_tuples([(i, 'Total') for i in totals.index])
# by_passer = pd.concat([by_passer, totals]).sort_index()
by_passer['Attempts'] = by_passer['Attempts'] - by_passer['Sacks']

by_passer['Yds / Att'] = round(by_passer['Yards'] / by_passer['Attempts'], 2)
by_passer['Success Rate'] = round((by_passer['Successes'] / by_passer['Plays']) * 100, 2)
by_passer['EPA / Play'] = round((by_passer['EPA'] / by_passer['Plays']), 2)
by_passer['1D Rate'] = round((by_passer['FirstDowns'] / by_passer['Attempts']) * 100, 2)
by_passer['TD Rate'] = round((by_passer['TDs'] / by_passer['Attempts']) * 100, 2)

print(by_passer.to_string())

                   Plays  Attempts  Completions  Yards  TDs  INTs  Sacks  SackYards  FirstDowns  Successes        EPA  Yds / Att  Success Rate  EPA / Play    1D Rate  TD Rate
posteam passer                                                                                                                                                                
IND     D.Jones     59.0      49.0         31.0  342.0  1.0   3.0    5.0      -29.0        19.0       27.0 -15.840039       6.98     45.759998       -0.27  38.779999     2.04
PIT     A.Rodgers   40.0      35.0         25.0  203.0  1.0   0.0    3.0      -16.0        11.0       18.0  -3.165599       5.80     45.000000       -0.08  31.430000     2.86


## Receiving

In [11]:
''' Receiving '''
# NOTE - 

by_receiver = pass_data.groupby(['posteam', 'receiver']).aggregate(
    Plays=('pass', 'sum'),
    Targets=('pass_attempt', 'sum'),
    Receptions=('complete_pass', 'sum'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('success', 'sum'),
    EPA=('epa', 'sum'),
).sort_values(by=['posteam', 'Targets'], ascending=False)

by_receiver['Yds / Rec'] = round(by_receiver['Yards'] / by_receiver['Receptions'], 2)
by_receiver['Success Rate'] = round((by_receiver['Successes'] / by_receiver['Plays']) * 100, 2)
by_receiver['EPA / Play'] = round((by_receiver['EPA'] / by_receiver['Plays']), 2)
by_receiver['1D Rate'] = round((by_receiver['FirstDowns'] / by_receiver['Plays']) * 100, 2)
by_receiver['TD Rate'] = round((by_receiver['TDs'] / by_receiver['Plays']) * 100, 2)

print(by_receiver.to_string())

                      Plays  Targets  Receptions  Yards  TDs  FirstDowns  Successes       EPA  Yds / Rec  Success Rate  EPA / Play    1D Rate  TD Rate
posteam receiver                                                                                                                                      
PIT     C.Austin        6.0      6.0         5.0   56.0  0.0         2.0        4.0  3.574144      11.20     66.669998        0.60  33.330002     0.00
        D.Washington    6.0      6.0         4.0   43.0  0.0         3.0        4.0 -0.207770      10.75     66.669998       -0.03  50.000000     0.00
        J.Smith         5.0      5.0         3.0   25.0  0.0         1.0        3.0  0.494034       8.33     60.000000        0.10  20.000000     0.00
        K.Gainwell      6.0      5.0         4.0   35.0  0.0         3.0        3.0  1.662904       8.75     50.000000        0.28  50.000000     0.00
        D.Metcalf       4.0      4.0         2.0    6.0  0.0         0.0        0.0 -2.160772 

In [277]:
''' Receiving by Down / Distance '''

DOWN_COL = 'Down & Distance' #'down'

## Group by down/distance
receiver_Plays = pass_data.groupby(['receiver', DOWN_COL]).aggregate(
    Plays=('pass', 'sum'),
    Successes=('success', 'sum')
).reset_index()
receiver_Plays['Success Rate'] = receiver_Plays['Successes'] / receiver_Plays['Plays']

# receiver_Plays = receiver_Plays.loc[receiver_Plays['Plays'] > 1, :]

## Pivot
DOWNS = [1,2,3,4]
DOWNS_SUBGROUP = ['1st', '2nd & Short', '2nd & Medium', '2nd & Long', 
       '3rd & Short', '3rd & Medium', '3rd & Long', '4th & Short', '4th & Medium']
col_sort = pd.MultiIndex.from_product([DOWNS_SUBGROUP, ['Successes', 'Plays', 'Success Rate']])

piv = receiver_Plays.loc[(receiver_Plays[DOWN_COL] != ''),:].pivot(
        index='receiver',
        columns=[DOWN_COL],
        values=['Plays', 'Successes', 'Success Rate']
    )\
    .swaplevel(0, 1, axis=1).sort_index(axis=1).reset_index().set_index('receiver')\
    .reindex(columns=col_sort)

print(piv.to_string())

## Visualize Success Rate

# Filter to success rate
idx = pd.IndexSlice
sl = pd.DataFrame(piv.loc[:, idx[:, ('Success Rate')]])
sl.columns = sl.columns.get_level_values(0)

# Merge in total Plays
sl = sl.merge(
        by_receiver.reset_index().set_index('receiver')['Plays'], 
        right_index=True,
        left_index=True,
        how='left'
    ).sort_values(by='Plays', ascending=False)\
    .reset_index().set_index('receiver')
# piv = piv.set_index('receiver')
# print(piv.to_string())

# Create annotations
annot_df = sl.copy() # Initialize with data, convert to string
for r in annot_df.index:
    for c in piv.columns.get_level_values(0).unique():
        plays = piv.loc[r,(c, 'Plays')]
        if pd.isna(plays):
            annot_df.loc[r,c] = ''
        else:
            annot_df.loc[r, c] = f"{piv.loc[r, (c, 'Success Rate')]*100:,.1f}%<br>({piv.loc[r,(c, 'Successes')]:,.0f} / {plays:,.0f})"

# print(annot_df.head().to_string())
# print(sl.head().to_string())
# print(sl.index)

# Heatmap

cols = [col.replace(' ', '<br>') for col in sl.columns.get_level_values(0).drop('Plays')]
# cols = ['1st', '2nd', '3rd', '4th']

fig = heat_map(
    x=cols,
    y=sl.index,
    z=sl.values.tolist(),
    text=annot_df.values.tolist(),
    title=f'Receiving <span style="color: #006D2C">Success Rate</span> by {DOWN_COL}<br><sup>NFL Week {WEEK}: {AWAY_TEAM} @ {HOME_TEAM}</sup>',
    color_name='Success Rate'
)
fig.update_layout(
    height=900
)
fig.show()

# pio.write_image(fig, "Success Rate by Down & Distance - Colts vs. Rams.png", scale=6)


                 1st                    2nd & Short                    2nd & Medium                    2nd & Long                    3rd & Short                    3rd & Medium                    3rd & Long                    4th & Short                    4th & Medium                   
           Successes Plays Success Rate   Successes Plays Success Rate    Successes Plays Success Rate  Successes Plays Success Rate   Successes Plays Success Rate    Successes Plays Success Rate  Successes Plays Success Rate   Successes Plays Success Rate    Successes Plays Success Rate
receiver                                                                                                                                                                                                                                                                                        
A.Abdullah       NaN   NaN          NaN         NaN   NaN          NaN          NaN   NaN          NaN        NaN   NaN          NaN 

# Players

In [12]:

passer_sl = by_passer[['Plays', 'EPA', 'EPA / Play']].copy().rename_axis(index={'passer': 'player'})
passer_sl.columns = ['Passing ' + col for col in passer_sl.columns]

receiver_sl = by_receiver[['Plays', 'EPA', 'EPA / Play']].copy().rename_axis(index={'receiver': 'player'})
receiver_sl.columns = ['Receiving ' + col for col in receiver_sl.columns]

rusher_sl = by_rusher[['Plays', 'EPA', 'EPA / Play']].copy().rename_axis(index={'rusher': 'player'})
rusher_sl.columns = ['Rushing ' + col for col in rusher_sl.columns]

## Combine ##
player_epa = passer_sl.merge(receiver_sl, left_index=True, right_index=True, how='outer')
player_epa = player_epa.merge(rusher_sl, left_index=True, right_index=True, how='outer')
player_epa = player_epa.fillna(0)

player_epa['Total Plays'] = player_epa['Passing Plays'] + player_epa['Receiving Plays'] + player_epa['Rushing Plays']
player_epa['Total EPA'] = player_epa['Passing EPA'] + player_epa['Receiving EPA'] + player_epa['Rushing EPA']
player_epa = player_epa.sort_values(by='Total EPA', ascending=False)

player_epa['team_logo_espn'] = player_epa.index.get_level_values(0).map(team_data['team_logo_espn'])
player_epa['team_color'] = player_epa.index.get_level_values(0).map(team_data['team_color'])
# player_epa['headshot'] = player_epa.index.get_level_values(1).map(player_info['headshot'])

print(player_epa.to_string())

                      Passing Plays  Passing EPA  Passing EPA / Play  Receiving Plays  Receiving EPA  Receiving EPA / Play  Rushing Plays  Rushing EPA  Rushing EPA / Play  Total Plays  Total EPA                                     team_logo_espn team_color
posteam player                                                                                                                                                                                                                                                  
IND     M.Pittman               0.0     0.000000                0.00             13.0       4.283083                  0.33            0.0     0.000000                0.00         13.0   4.283083  https://a.espncdn.com/i/teamlogos/nfl/500/ind.png    #002C5F
PIT     C.Austin                0.0     0.000000                0.00              6.0       3.574144                  0.60            0.0     0.000000                0.00          6.0   3.574144  https://a.espncdn.com/i/teamlogos

In [14]:

## Data ##
x=player_epa['Total EPA'].tolist()
y=player_epa.index.get_level_values(1)
logos = player_epa['team_logo_espn'].tolist()
colors = player_epa['team_color'].tolist()

n_players = len(x)


## Bar ##
bar = px.bar(
    x=x,
    y=y
)

fig = go.Figure()

for trace in bar.data:
    fig.add_trace(trace)

# Logos

for i in range(len(logos)):
    fig.add_layout_image(
        x=-0.1,
        y=(1-((1/n_players)*.5))-((1/(n_players))*i),
        sizex=(1/n_players)*.9,
        sizey=(1/n_players)*.9,
        xanchor='center',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=logos[i],
    )

# Format
fig.update_traces(marker=dict(opacity=0.7, color=colors, line=dict(width=1.5, color='black')))
fig.update_yaxes(
    autorange='reversed'
)
fig.update_xaxes(
    title='Total EPA'
)
fig.update_layout(
    template='nfl_template',
    title=dict(text=f'Player EPA Contributions<br><sup>Week {WEEK}: {AWAY_TEAM} @ {HOME_TEAM}</sup>'),
    height=500, width=900,
    margin=dict(t=75, r=25, b=65, l=100)
)

# Credits
fig.add_annotation(
    text=f'Total EPA on non-special teams plays; passing + receiving + rushing<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.15, 
    x=1,
    align='right'
)

fig.show()

# Export
pio.write_image(fig, f'Player EPA Contributions - Week {WEEK} - {AWAY_TEAM} @ {HOME_TEAM}.png', scale=6, width=900, height=500)

In [ ]:

## Bar ##
teams = [AWAY_TEAM, HOME_TEAM]
wordmarks = [team_data.loc[team_data.index == AWAY_TEAM, 'team_wordmark'].values[0], team_data.loc[team_data.index == HOME_TEAM, 'team_wordmark'].values[0]]
units = ['Passing', 'Receiving', 'Rushing']

bar_charts = []
x_ranges = []
for unit in units:
    plays_col = f'{unit} Plays'
    epa_col = f'{unit} EPA'

    unit_sl = player_epa.loc[player_epa[plays_col] > 0, :].copy()
    unit_sl = unit_sl.sort_values(by=plays_col, ascending=False)

    r = abs(unit_sl[epa_col].max() - unit_sl[epa_col].min())
    epa_range = [unit_sl[epa_col].min() - (r*.2), unit_sl[epa_col].max() + (r*.2)]
    if epa_range[0] > 0:
        epa_range[0] = 0 - (r*.2)
    if epa_range[1] < 0:
        epa_range[1] = 0 + (r*.2)

    x_ranges.append(epa_range)

    for team in teams:
        team_sl = unit_sl.loc[unit_sl.index.get_level_values(0) == team, :]

        x = team_sl[epa_col].tolist()
        y = team_sl.index.get_level_values(1)
        text = team_sl[plays_col].tolist()
        colors = team_sl['team_color'].tolist()

        bar = px.bar(
            x=x,
            y=y,
            text=text,
            range_x=epa_range,
            color_discrete_sequence=colors,
            opacity=0.7
        )

        bar_charts.append(bar)


## Create Figure ##
N_ROWS = 3
N_COLS = 2

H_SPACING = .25/N_COLS
V_SPACING = .2/N_ROWS
fig = make_subplots(rows=N_ROWS, cols=N_COLS, 
                    row_heights=[1,4,3], column_widths=[1,1],
                    horizontal_spacing=H_SPACING, vertical_spacing=V_SPACING,
                    row_titles=units, x_title='Total EPA')

fig.for_each_annotation(lambda a: a.update(x=-.135, textangle=-90, font=dict(weight='bold', size=14)) if a.text in units else())
fig.for_each_annotation(lambda a: a.update(y=.015, font=dict(weight='bold', size=14)) if a.text == 'Total EPA' else())

# Add bars
i = 0
for row in range(1, N_ROWS+1):
    for col in range(1, N_COLS+1):
        bar = bar_charts[i]
        
        for trace in bar.data:
            fig.add_trace(trace, row=row, col=col)

        i += 1

# Wordmarks
for i in range(len(wordmarks)):
    response = requests.get(wordmarks[i])
    logo_img = Image.open(BytesIO(response.content))

    col_wid = (1 / N_COLS) - (H_SPACING / 2)
    fig.add_layout_image(
        x=(col_wid / 2) if i == 0 else 1 - (col_wid / 2),
        y=1.05,
        sizex=.2,
        sizey=.2,
        xanchor='center',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=logo_img,
    )

# Format
fig.update_traces(marker=dict(opacity=0.7, line=dict(width=2, color='#323232')))
fig.update_yaxes(
    autorange='reversed',
)
for i in range(N_ROWS):
    r_val = x_ranges[i][1] - x_ranges[i][0]
    fig.update_xaxes(range=x_ranges[i], dtick=r_val // 7, row=i+1)


# Credits
fig.add_annotation(
    text=f'Sorted by number of plays (bar text)<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.1, 
    x=1,
    align='right'
)

fig.update_layout(
    template='nfl_template',
    title=dict(text=f'Player EPA Contributions<br><sup>Week {WEEK}: {AWAY_TEAM} @ {HOME_TEAM}</sup>'),
    height=700, width=700,
    margin=dict(t=100, r=15, b=65, l=90)
)

fig.show()

# Export
# pio.write_image(fig, f'Player EPA Contributions - Week {WEEK} - {AWAY_TEAM} @ {HOME_TEAM}.png', scale=6, width=700, height=700)

-15.840039 -3.165599
range: 12.67444
[-18.37492733001709, -0.6307110309600827]
here: 2.534888076782227
-3.8755553 4.2830834
range: 8.158639
[-5.507283067703247, 5.91481122970581]
-4.220659 4.875206
range: 9.095865
[-6.039831829071045, 6.694379043579102]


## EPA

In [ ]:
''' EPA by Play Type '''

epa_summary = game_data.groupby(['posteam', 'play_type']).aggregate({

    'posteam': 'size', 
    'ep': 'sum',
    'epa': 'sum'
}).rename(columns={'posteam': 'Plays'})
epa_summary['EPA / Play'] = epa_summary['epa'] / epa_summary['Plays']

print(epa_summary.groupby('posteam')['epa'].sum())
print(epa_summary.to_string())

posteam
IND    19.897320
TEN    -8.311419
Name: epa, dtype: float32
                     Plays         ep        epa  EPA / Play
posteam play_type                                           
IND     extra_point      5   4.657144   0.342857    0.068571
        field_goal       1   2.150450   0.849550    0.849550
        kickoff          3   4.185029   0.437633    0.145878
        no_play          1   2.011646  -1.522343   -1.522343
        pass            34  86.031448  11.726611    0.344900
        punt             4  -3.040758  -0.050828   -0.012707
        run             18  34.533447   8.113840    0.450769
TEN     extra_point      2   1.862857   0.137143    0.068571
        field_goal       1   1.626989  -1.949176   -1.949176
        kickoff          7   7.709158  -0.099621   -0.014232
        no_play          3   2.118842   3.248645    1.082882
        pass            41  75.061142  -8.368195   -0.204102
        punt             5  -2.842280  -3.875555   -0.775111
        run      

In [87]:
''' Drives EPA '''

epa_summary = game_data.groupby(['posteam', 'drive']).aggregate({
    'posteam': 'size', 
    'ep': 'first',
    'epa': 'sum'
}).rename(columns={'posteam': 'Plays', 'ep': 'Start EP'})
epa_summary['Drive End EP'] = epa_summary['Start EP'] + epa_summary['epa']
epa_summary['Drive Exp. Pts'] = np.where(epa_summary['Drive End EP'] < 0, 0, epa_summary['Drive End EP'])
epa_summary['EPA / Play'] = epa_summary['epa'] / epa_summary['Plays']

print(epa_summary.to_string())
print(epa_summary.groupby('posteam')['Drive Exp. Pts'].sum())

               Plays  Start EP       epa  Drive End EP  Drive Exp. Pts  EPA / Play
posteam drive                                                                     
IND     1.0       11  1.592993  1.407007      3.000000        3.000000    0.127910
        3.0        9  1.540722  5.527849      7.068571        7.068571    0.614205
        5.0        8  1.472162  5.596409      7.068572        7.068572    0.699551
        7.0        4  1.967553 -2.827820     -0.860267        0.000000   -0.706955
        9.0        5  0.542682 -0.781268     -0.238586        0.000000   -0.156254
        11.0       1  0.322187 -0.322187      0.000000        0.000000   -0.322187
        13.0       2  1.113361  5.955210      7.068572        7.068572    2.977605
        15.0      10  1.553515  5.515056      7.068572        7.068572    0.551506
        17.0       7  3.220508  3.848063      7.068571        7.068571    0.549723
        19.0       4  0.908393 -2.543533     -1.635140        0.000000   -0.635883
    

In [88]:
''' Best and Worst EPA Plays '''


for side in ['away', 'home']:
    col = 'home_team' if side == 'home' else 'away_team'
    team = game_data.loc[game_data.first_valid_index(), col]
    
    plays = game_data.loc[game_data['posteam_type'] == side, ['epa', 'qtr', 'time', 'yrdln', 'down', 'ydstogo', 'desc']].sort_values(by='epa', ascending=False)

    print(f'{team}')
    print('Best plays:')
    print(plays.head().to_string(index=False))
    print('Worst plays:')
    print(plays.tail().to_string(index=False))


TEN
Best plays:
     epa  qtr  time  yrdln  down  ydstogo                                                                                                                                                                                                                                 desc
4.529117  2.0 12:05 IND 42   3.0      6.0                                                                                                                                                  (12:05) (Shotgun) 2-T.Spears right end pushed ob at IND 1 for 41 yards (0-C.Bynum).
3.846205  4.0 02:40  IND 3   4.0      2.0                                                                                                                                                                                (2:40) 2-T.Spears left tackle for 3 yards, TOUCHDOWN.
3.186444  3.0 03:16 TEN 44   4.0      6.0 (3:16) (Shotgun) 1-C.Ward pass incomplete short middle to 11-V.Jefferson [97-L.Latu]. PENALTY on IND-23-K.Moore, Defensive Holdin